# Import

In [14]:
import pandas as pd
from pathlib import Path
import os

# Data
Read and file `NaN` values with `0`

In [ ]:
raw = pd.read_csv('./data/MyExpt_Image(in).csv', na_values='nan',).fillna(0)

# Create compressed data
Sort values by `Series_G3BP` and `Metadata_T` and extract columns

In [3]:
extract_cols = ['Series_G3BP', 'Metadata_T', 'Count_G3BP_Puncta', 'Count_Nuclei', 'Mean_G3BP_Puncta_AreaShape_Area', 'Mean_G3BP_Puncta_Intensity_MeanIntensity_G3BP', 'Mean_Nuclei_AreaShape_Area']
compressed = raw.sort_values(['Series_G3BP', 'Metadata_T']).loc[:,extract_cols]

compressed['Total_SG_Area'] = compressed['Count_G3BP_Puncta'] * compressed['Mean_G3BP_Puncta_AreaShape_Area']
compressed['Total_SG_Signal'] = compressed['Total_SG_Area'] * compressed['Mean_G3BP_Puncta_Intensity_MeanIntensity_G3BP']
compressed['Total_Nuc_Area'] = compressed['Mean_Nuclei_AreaShape_Area'] * compressed['Count_Nuclei']
compressed.reset_index(drop=True, inplace=True)

# Create sheets

In [ ]:
def make_df(compressed, column_labels, series_starts, continuous_index=0, s_idx=0):
    '''
    Parameters
    ----------
    compressed : pd.DataFrame()
        - a pandas DataFrame with the columns extracted from original data
    column_labels : list
        - a list of columns labels to extract from compressed
    series_starts : list
        - a list of start indexes for new series within the data for the Series_G3BP column
    continous_index : int
        - used for tracking with compressed
    s_idx : int
        - used for tracking index of series_starts
    
    Returns
    -------
    df : pd.DataFrame()
        - a pandas DataFrame with normalized data from compressed
    continous_index : int
        - used for tracking with compressed; returned int for next group
    s_idx : int
        - used for tracking index of series_starts; returned into for next group
    '''
    df = pd.DataFrame(columns=column_labels)
    # go through all raw
    for i in range(continuous_index, len(compressed)):
        # see if Series_G3BP is equal to or greater than series_start
        if compressed.loc[i,'Series_G3BP'] >= series_starts[s_idx]:
            # see if we can check next series_start
            if s_idx <= len(series_starts)-1:
                # check next series_start
                if compressed.loc[i,'Series_G3BP'] < (series_starts[s_idx]+(series_starts[1]-series_starts[0])): # would prefer to have < series_start[s_idx+1] for the edge case that there are not n0-n9 in the final group, or if there aren't 10 groups in the future
                    # construct dataframe
                    df.loc[i-continuous_index, 'Series_G3BP'] = compressed.loc[i, 'Series_G3BP']
                    df.loc[i-continuous_index, 'Metadata_T'] = compressed.loc[i, 'Metadata_T']
                    df.loc[i-continuous_index, 'Count_G3BP_Puncta'] = compressed.loc[i, 'Count_G3BP_Puncta']
                    df.loc[i-continuous_index, 'Count_Nuclei'] = compressed.loc[i, 'Count_Nuclei']
                    df.loc[i-continuous_index, 'Total_SG_Area'] = compressed.loc[i, 'Total_SG_Area']
                    df.loc[i-continuous_index, 'Total_SG_Signal'] = compressed.loc[i, 'Total_SG_Signal']
                    df.loc[i-continuous_index, 'Total_Nuc_Area'] = compressed.loc[i, 'Total_Nuc_Area']
                else:
                    continuous_index = i
                    s_idx += 1
                    break
            else: 
                print('ERROR: s_idx > len(series_starts) @ '+str(i)+' with s_idx='+str(s_idx))
                break
        else:
            print('ERROR: Series_G3BP below series_start')
            break
    else:
        continuous_index=i+1
    
    # norm by max
    max_by_series = df.groupby('Series_G3BP').max()
    for i in range(len(df)):
        i_max = df.loc[i,'Series_G3BP']
        df.loc[i, 'Count_G3BP_Puncta_norm_max'] = df.loc[i, 'Count_G3BP_Puncta']*100/max_by_series.loc[i_max,'Count_G3BP_Puncta']
        df.loc[i, 'Count_Nuclei_norm_max'] = df.loc[i, 'Count_Nuclei']*100/max_by_series.loc[i_max,'Count_Nuclei']
        df.loc[i, 'Total_SG_Area_norm_max'] = df.loc[i, 'Total_SG_Area']*100/max_by_series.loc[i_max,'Total_SG_Area']
        df.loc[i, 'Total_SG_Signal_norm_max'] = df.loc[i, 'Total_SG_Signal']*100/max_by_series.loc[i_max,'Total_SG_Signal']
        df.loc[i, 'Total_Nuc_Area_norm_max'] = df.loc[i, 'Total_Nuc_Area']*100/max_by_series.loc[i_max,'Total_Nuc_Area']

    # norm by nuclei count
    for i in range(len(df)):
        df.loc[i, 'Count_G3BP_Puncta_norm_count_nuclei'] = df.loc[i, 'Count_G3BP_Puncta']/compressed.loc[continuous_index-len(df)+i,'Count_Nuclei']
        df.loc[i, 'Count_Nuclei_norm_count_nuclei'] = df.loc[i, 'Count_Nuclei']/compressed.loc[continuous_index-len(df)+i,'Count_Nuclei']
        df.loc[i, 'Total_SG_Area_norm_count_nuclei'] = df.loc[i, 'Total_SG_Area']/compressed.loc[continuous_index-len(df)+i,'Count_Nuclei']
        df.loc[i, 'Total_SG_Signal_norm_count_nuclei'] = df.loc[i, 'Total_SG_Signal']/compressed.loc[continuous_index-len(df)+i,'Count_Nuclei']
        df.loc[i, 'Total_Nuc_Area_norm_count_nuclei'] = df.loc[i, 'Total_Nuc_Area']/compressed.loc[continuous_index-len(df)+i,'Count_Nuclei']

    # norm by nuclear area
    for i in range(len(df)):
        df.loc[i, 'Count_G3BP_Puncta_norm_nuc_area'] = df.loc[i, 'Count_G3BP_Puncta']*100/compressed.loc[continuous_index-len(df)+i,'Total_Nuc_Area']
        df.loc[i, 'Count_Nuclei_norm_nuc_area'] = df.loc[i, 'Count_Nuclei']*100/compressed.loc[continuous_index-len(df)+i,'Total_Nuc_Area']
        df.loc[i, 'Total_SG_Area_norm_nuc_area'] = df.loc[i, 'Total_SG_Area']*100/compressed.loc[continuous_index-len(df)+i,'Total_Nuc_Area']
        df.loc[i, 'Total_SG_Signal_norm_nuc_area'] = df.loc[i, 'Total_SG_Signal']*100/compressed.loc[continuous_index-len(df)+i,'Total_Nuc_Area']
        df.loc[i, 'Total_Nuc_Area_norm_nuc_area'] = df.loc[i, 'Total_Nuc_Area']*100/compressed.loc[continuous_index-len(df)+i,'Total_Nuc_Area']

    return df, continuous_index, s_idx

# Run all labels

In [ ]:
# Gives the start value for each group in `Series_G3BP` that is used to create groups `JRR_UT`, `TGM_UT`, `JRR_250`, `TGM_250`, `JRR_500`, `TGM_500`. 
# Assumes consecutive values to make group (i.e. 0-9 = `JRR_UT`)
series_starts = [0, 10, 20, 30, 40, 50]
# group names corresponding to index starts
groups = ['JRR_UT', 'TGM_UT', 'JRR_500', 'TGM_500', 'JRR_250', 'TGM_250']
# columns used in the final spreadsheet
make_cols = ['Series_G3BP', 'Metadata_T', 'Count_G3BP_Puncta', 'Count_Nuclei', 'Total_SG_Area',  'Total_SG_Signal', 'Total_Nuc_Area'] # columns made each time
# File naming scheme
file_prefix = '20250521_Video_Analysis_'
file_suffix = '_nomalized_test'
excel_file_name = file_prefix+file_suffix[1:]+'.xlsx'

# run
# delete existing data so we can append df as new sheet, otherwise new xlsx files are created each time
if os.path.exists('./output/'+excel_file_name):
    print('Deleting Data')
    os.remove('./output/'+excel_file_name)

# write data
continuous_index = 0
s_idx = 0
for group in groups:
    print('Writing Group: '+group)
    df, continuous_index, s_idx = make_df(compressed, make_cols, series_starts, continuous_index, s_idx)
    # df.to_csv('./output/'+file_prefix+group+file_suffix+'.csv', index=False)

    if os.path.exists('./output/'+excel_file_name):
        # this engine allows for appending to an existing xlsx file (but can't create one)
        with pd.ExcelWriter('./output/'+excel_file_name, mode='a', engine='openpyxl') as writer:
            df.to_excel(writer, sheet_name=group, index=False)
    else:
        # this engine can create and then write to the xlsx file (but can't append to one)
        with pd.ExcelWriter('./output/'+excel_file_name, mode='w', engine='xlsxwriter') as writer:
            df.to_excel(writer, sheet_name=group, index=False)

print('Done')    

Deleting Data
Writing Group: JRR_UT
Writing Group: TGM_UT
Writing Group: JRR_500
Writing Group: TGM_500
Writing Group: JRR_250
Writing Group: TGM_250
Done


# Verification